In [1]:
import logging
from lisa.conf import LisaLogging
LisaLogging.setup()

2018-09-18 16:01:16,645 INFO    : root         : Using LISA logging configuration:
2018-09-18 16:01:16,646 INFO    : root         :   /home/dourai01/Data/Git/lisa/py3k/logging.conf


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%pylab inline

import json
import os

# Support to access the remote target
import devlib
from lisa.env import TestEnv

# Import support for Android devices
from lisa.android import Screen, Workload

# Support for trace events analysis
from lisa.trace import Trace

# Suport for FTrace events parsing and visualization
import trappy
import pandas as pd

from lisa.wlgen.rta import RTA, Periodic, Ramp
from time import sleep

from IPython.display import display

Populating the interactive namespace from numpy and matplotlib


In [4]:
linux_hikey = {
    "platform" : "linux",
    "board" : "hikey",
    "host": "pwrsft-hikey620-1",
    #"username" : "root",
    #"password" : "root",
    "modules" : ["sched", "cgroups", "hotplug"],
    "tools" : ["taskset", "rt-app"],
    "rtapp-calib" :  {
        "0": 302, "1": 302, "2": 302, "3": 302, "4": 136, "5": 136, "6": 136, "7": 136
     }
}

In [ ]:
linux_juno = {
    "platform" : "linux",
    "board" : "juno",
    "host": "192.168.0.1",
    "username" : "root",
    "password" : "root",
    "modules" : ["sched", "cgroups", "hotplug"],
    "tools" : ["taskset", "rt-app"]
}

In [ ]:
android_hikey = {
    "platform" : "android",
    "board" : "hikey960",
    "modules" : ["sched", "hotplug"],
    "tools" : ["taskset", "rt-app"]
}

In [5]:
te = TestEnv(linux_hikey)
target = te.target

2018-09-18 16:01:24,081 INFO    : TestEnv      : Using base path: /home/dourai01/Data/Git/lisa/py3k
2018-09-18 16:01:24,082 INFO    : TestEnv      : Loading custom (inline) target configuration
2018-09-18 16:01:24,083 INFO    : TestEnv      : Devlib modules to load: ['sched', 'hotplug', 'cpufreq', 'cgroups', 'cpuidle']
2018-09-18 16:01:24,083 INFO    : TestEnv      : Connecting linux target:
2018-09-18 16:01:24,084 INFO    : TestEnv      :   username : root
2018-09-18 16:01:24,084 INFO    : TestEnv      :   password : 
2018-09-18 16:01:24,085 INFO    : TestEnv      :       host : pwrsft-hikey620-1
2018-09-18 16:01:24,086 INFO    : TestEnv      : Connection settings:
2018-09-18 16:01:24,086 INFO    : TestEnv      :    {'username': 'root', 'password': '', 'host': 'pwrsft-hikey620-1'}
2018-09-18 16:01:29,183 INFO    : TestEnv      : Initializing target workdir:
2018-09-18 16:01:29,184 INFO    : TestEnv      :    /root/devlib-target
2018-09-18 16:01:29,979 INFO    : CGroups      : Availabl

### Generics

In [10]:
from lisa.tests.kernel.scheduler.eas_behaviour import *

In [11]:
tests = [OneSmallTask]#, ThreeSmallTasks, TwoBigTasks, TwoBigThreeSmall, RampUp, RampDown, EnergyModelWakeMigration]

In [12]:
res = {}

for test in tests:
    bundle = test.from_target(te)
    res[test.__name__] = bundle

2018-09-18 16:45:57,309 INFO    : TestEnv      : Using configuration provided RTApp calibration
2018-09-18 16:45:57,310 INFO    : TestEnv      : Using RT-App calibration values:
2018-09-18 16:45:57,312 INFO    : TestEnv      :    {"0": 302, "1": 302, "2": 302, "3": 302, "4": 136, "5": 136, "6": 136, "7": 136}
2018-09-18 16:45:57,707 INFO    : RTA          : Calibration value: 136
2018-09-18 16:45:57,709 INFO    : RTA          : Default policy: SCHED_OTHER
2018-09-18 16:45:57,710 INFO    : RTA          : ------------------------
2018-09-18 16:45:57,711 INFO    : RTA          : task [small], sched: using default policy
2018-09-18 16:45:57,713 INFO    : RTA          :  | start delay: 0.000000 [s]
2018-09-18 16:45:57,715 INFO    : RTA          :  | loops count: 1
2018-09-18 16:45:57,717 INFO    : RTA          :  + phase_000001
2018-09-18 16:45:57,718 INFO    : Phase        :  | duration 1.000000 [s] (62 loops)
2018-09-18 16:45:57,718 INFO    : Phase        :  |  period    16000 [us], duty_

In [14]:
for test_name, bundle in res.items():
    print(test_name)
    print("---")
    print("test_slack: {}".format("PASSED" if bundle.test_slack() else "FAILED"))
    print("test_task_placement: {}".format("PASSED" if bundle.test_task_placement() else "FAILED"))
    print(bundle.test_task_placement())
    print()

OneSmallTask
---
test_slack: PASSED


2018-09-18 16:48:54,019 WARNING : Trace        : Kernel version not available from platform data
2018-09-18 16:48:54,019 INFO    : Trace        : Parsing trace assuming kernel v3.18


test_task_placement: PASSED
estimated_energy = 229.98293056713615 bogo-joules, energy_threshold = 245.74051446945342 bogo-joules



### CPU sanity check

In [16]:
from lisa.tests.kernel.misc.capacity_sanity import CapacitySanityCheck

In [17]:
bundle = CapacitySanityCheck.from_target(te)

2018-09-18 16:49:29,986 INFO    : Sysbench     : Execution start: /root/devlib-target/bin/taskset 0x1 /root/devlib-target/bin/sysbench --test=cpu --max-time=1 run
2018-09-18 16:49:31,165 INFO    : Sysbench     : Execution complete
2018-09-18 16:49:31,498 INFO    : Sysbench     : Execution start: /root/devlib-target/bin/taskset 0x2 /root/devlib-target/bin/sysbench --test=cpu --max-time=1 run
2018-09-18 16:49:32,675 INFO    : Sysbench     : Execution complete
2018-09-18 16:49:33,008 INFO    : Sysbench     : Execution start: /root/devlib-target/bin/taskset 0x4 /root/devlib-target/bin/sysbench --test=cpu --max-time=1 run
2018-09-18 16:49:34,186 INFO    : Sysbench     : Execution complete
2018-09-18 16:49:34,519 INFO    : Sysbench     : Execution start: /root/devlib-target/bin/taskset 0x8 /root/devlib-target/bin/sysbench --test=cpu --max-time=1 run
2018-09-18 16:49:35,698 INFO    : Sysbench     : Execution complete
2018-09-18 16:49:36,032 INFO    : Sysbench     : Execution start: /root/devl

In [18]:
if bundle.test_capacity_sanity():
    print("PASSED")
else:
    print("FAILED")

print(bundle.test_capacity_sanity())

PASSED
Performance @1024 capacity = 227


### Hotplug

In [20]:
from lisa.tests.kernel.hotplug.torture import HotplugTorture

In [ ]:
bundle = HotplugTorture.from_target(te)

In [ ]:
bundle.test_cpus_alive().passed

In [ ]:
bundle.test_cpus_alive().metrics

In [ ]:
bundle.test_target_alive().passed